In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import shutil

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

#name_class kendi siniflariniza gore duzenlemelisiniz.
name_class = ['Durak','Girilmez','Kirmizi_Isik','Yesil_Isik','Park_Yapilmaz','Saga_Donulmez','Sola_Donulmez'] 
counter = np.zeros(len(name_class))
validation_test = 10

def bol(file_name):
    _, name= file_name.split('/')
    for i in range(len(name_class)):
        if name_class[i] == name[:-5] or name_class[i] == name[:-6] :
            counter[i] += 1
            if counter[i] % validation_test == 0:
                shutil.move(file_name,'images/test/')
            else :
                shutil.move(file_name,'images/train/')

            break

def main():
    for file_name in glob.glob('images/*.xml'):
        print file_name
        bol(file_name)

    for directory in ['train','test']:
        image_path = os.path.join(os.getcwd(), 'images/{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')

main()